In [ ]:
import os
from D_Search.PDFMiner import PDFMiner
from A_Configuration_and_Logs.conf_and_log import ConfLog
from E_Collect.Collect import get_values_and_page_numbers

keyword_dict_of_lists = {'Scope 1': ['Scope 1', 'scope 1'], 'Scope 1 and 2': ['Scope 1 and 2', 'scope 1 and 2']}
search_word_list = ['tCO2e', '(t) CO2e', 't CO2', 't CO₂', 'tons CO2', 'tons CO₂', 't CO₂e', 'CO₂e', 'CO2e',  'CO2 equivalent', 'CO₂ equivalent']


conf_log = ConfLog()
for orig_pdf in os.scandir(conf_log.path_to_reports_for_analysis_directory):
    filename = os.fsdecode(orig_pdf)
    if filename.endswith(".pdf"):
        try:
            miner = PDFMiner(path=filename)
            table_keywords = miner.get_year_and_fy()
            search_result = miner.find_word(keywords_dict_of_list=keyword_dict_of_lists,
                                            search_word_list=search_word_list,
                      table_keywords=table_keywords,neighbour_tolerance=1.50)
            table_numbers_and_pages = get_values_and_page_numbers(search_result_list=search_result, keyword_dict_of_lists=keyword_dict_of_lists,
                                table_keywords=table_keywords, search_result_dict_key_name='table_values')
            text_numbers_and_pages = get_values_and_page_numbers(search_result_list=search_result, keyword_dict_of_lists=keyword_dict_of_lists,
                                table_keywords=table_keywords, search_result_dict_key_name='short_text_and_number')
            print('filename:', filename)
            print('table_numbers_and_pages:', table_numbers_and_pages)
            print('text_numbers_and_pages:', text_numbers_and_pages)
            print('----------------------------------------')

            miner.stream.close()
        except Exception as e:
            conf_log.logging.error(e, exc_info=True)
